What I've done so far

In [1]:
import geopandas as gpd
from glob import glob
import dask.dataframe as da
from dask.distributed import LocalCluster, Client
import matplotlib.pyplot as plt
import os
import rioxarray as rio
import xarray as xr
# from dem_utils import ArcticDEM
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
from shapely import wkt
# import utils
import cartopy.crs as ccrs
import shapely
from rasterio.features import rasterize
from rasterio.enums import Resampling
import seaborn as sns
# import odc.geo.xr
from cycler import cycler
prj = ccrs.Stereographic(
    central_latitude=90,
    central_longitude=-45,
    true_scale_latitude=70
)


In [3]:
# helper functions
def demote_coords_to_vars(ds: xr.Dataset,
                        coords: str,
                        var_name: str):
    '''
    messy onliner to for reorganizing dataset.
    e.g. dataset with two variables: a (dims: x, y, t) and b (dims: x, y)
    this function will convert it to a dataset with 
    dimensions x, y and add as many `a` variables as there dim `t` is long
    '''
    return xr.merge([
        ds.drop_vars([coords, var_name]),
        xr.merge(
            [ds[var_name].isel({coords:i}).rename(ds[coords][i].item())
            for i in range(len(ds[coords]))], compat='override').drop_vars(coords)]
                    )

def add_geom_mask(ds, geom, buffer=200):
    # buffer geometry, with square ends
    buff_geom = geom.buffer(buffer, cap_style=3)
    
    # empty array of same x, y dim shape as merged
    arr = np.zeros((ds.sizes['y'], ds.sizes['x']))
    
    # rasterize
    burned = rasterize(shapes=[(buff_geom, 1)],
                       fill=0,
                       out=arr,
                       transform=ds.rio.transform())
    
    # merged rasterized with all other dataarrays
    merged = xr.merge([ds, xr.DataArray(data=burned,
                                        dims=['y','x'],
                                        coords={'y': ds.y,
                                                'x': ds.x}).rename('buffer_aoi')])

    return merged

class site():
    def __init__(self,
                 id: int,
                 vars: list=['sec', 'dem', 'sample', 'coreg_meta', 'stable_terrain', 'centreline']):
        
        '''
        convenience class for opening output files from directory id
        id = id number of study site directory
        vars = list of variables to include
        returns the opened files
        '''
        
        directories = glob('../data/id*')
        directory = [d for d in directories if f'id{id}' in d]
        assert len(directory) == 1, 'too many or not enough'
        self.directory = directory[0]

        self.paths = {
            'sec': os.path.join(self.directory, 'sec.zarr'),
            'dem': os.path.join(self.directory, 'stacked_coregd.zarr'),
            'sample': os.path.join(self.directory, 'sec_sample.parquet'),
            'coreg_meta': os.path.join(self.directory, 'coregistration_metadata.parquet'),
            'stable_terrain': os.path.join(self.directory, 'stable_terrain_mask.tif'),
            'centreline': os.path.join(self.directory, glob('line*.geojson', root_dir=self.directory)[0])
            }

        to_remove = []
        for k, v in self.paths.items():
            if os.path.exists(v):
                continue
            else:
                to_remove.append(k)
        
        [self.paths.pop(k) for k in to_remove]
                
        self.open_funcs = {
            '.tif': rio.open_rasterio,
            '.zarr': xr.open_zarr,
            '.parquet': pd.read_parquet,
            '.geojson': gpd.read_file
        }
        
        for var in [var for var in vars if var in self.paths.keys()]:
            _, extension = os.path.splitext(self.paths[var])
            setattr(self, var, self.open_funcs[extension](self.paths[var]))
        
        try:
            self.sec = demote_coords_to_vars(self.sec, 'result', 'sec')
        except:
            pass


## Surface elevation change

### meta data

In [60]:
# grab some metadata
site_directories = glob('../data/id*')
centrelines = gpd.read_file('../data/streams_v3.geojson')

dem_ids = []
dem_counts = []
useful_dem_counts = []
for d in site_directories:
    df= pd.read_parquet(
        os.path.join(d, 'coregistration_metadata.parquet')
        )
    dem_ids += df['to_reg_dem_id'].unique().tolist()
    dem_counts.append(len(df['to_reg_dem_id'].unique()))
    useful_dem_counts.append(((df['nmad_after'] < 2) & (df['median_after'].abs() < 1)).sum())

In [63]:
# show the meta data
print('number of study sites, by location and type')
print(centrelines.groupby(['where', 'lake_land'])['geometry'].count().to_markdown())

print(f'\ntotal of {len(dem_ids)} _sections_ of DEMs used and coregistered (total of: {len(set(dem_ids))} tiles)')
print(f'each of the 36 study sites on average used {np.mean(dem_counts):.0f}'
      f'(mean) {np.median(dem_counts):.0f} (median) DEMs.',
      '\n(where used means coregistered)\n')

print('Coregistered DEMs were removed from subsequent analyses if they '
      'failed to meet NMAD and MDOST of 2 m and ±1 m, respectively')
print(f'the site with the fewest/most DEMs used {np.min(useful_dem_counts):.0f}/{np.max(useful_dem_counts):.0f}. '
      f'mean/median: {np.mean(useful_dem_counts):.0f}/{np.median(useful_dem_counts):.0f}')

number of study sites, by location and type
|                       |   geometry |
|:----------------------|-----------:|
| ('greenland', 'lake') |         14 |
| ('greenland', 'land') |          9 |
| ('iceland', 'lake')   |          7 |
| ('iceland', 'land')   |          6 |

total of 2153 _sections_ of DEMs used and coregistered (total of: 1629 tiles)
each of the 36 study sites on average used 60(mean) 48 (median) DEMs. 
(where used means coregistered)

Coregistered DEMs were removed from subsequent analyses if they failed to meet NMAD and MDOST of 2 m and ±1 m, respectively
the site with the fewest/most DEMs used 2/148. mean/median: 48/36


### results